In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
pip install ktrain

In [3]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline    
import operator   
import tensorflow as tf
import random
from keras.preprocessing.image import ImageDataGenerator
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
import ktrain

In [4]:
DIRECTORY = r'D:\Leuk Dataset\new fol\train'
VAL_DIRECTORY = r'D:\Leuk Dataset\new fol\val'

CATEGORIES = ['all', 'hem']

In [5]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    return img[slices]

In [ ]:
data = []
i = 0
plt.figure(figsize=(15, 15))
for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path)
        crop_arr = crop_center(arr, (150,150))
        if 1 <= i+1 <= 140:                      # total 140 image
            ax = plt.subplot(13, 11, i+1)
        plt.imshow(crop_arr)
        i += 1
        data.append([crop_arr, label])
#         if i+1==3000:
#             break

In [ ]:
random.shuffle(data)

x_train = []
y_train = []

for features, label in data:
    x_train.append(features)
    y_train.append(label)

    
x_train = np.array(x_train)
y_train = np.array(y_train)
print(x_train.shape)

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')

x_train = x_train / 255.0
y_train = y_train / 255.0

# np.save("x_train_leuknet",x_train)
# np.save("y_train_leuknet",y_train)

In [ ]:
val_data = []
i = 0
plt.figure(figsize=(15, 15))
for category in CATEGORIES:
    path = os.path.join(VAL_DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path)
        crop_arr = crop_center(arr, (150,150))
        if 1 <= i+1 <= 70:                     # total image 70
            ax = plt.subplot(10, 7, i+1)
        plt.imshow(crop_arr)
        i += 1
        val_data.append([crop_arr, label])

In [ ]:
random.shuffle(val_data)

x_val = []
y_val = []

for features, label in val_data:
    x_val.append(features)
    y_val.append(label)

    
x_val = np.array(x_val)
y_val = np.array(y_val)

x_val = x_val.astype('float32')
y_val = y_val.astype('float32')

x_val = x_val / 255.0
y_val = y_val / 255.0

print(x_val.shape)
print(y_val.shape)

In [ ]:
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, GlobalAveragePooling2D
def get_model():
    base_model = tf.keras.applications.EfficientNetB0(
        include_top=False,
        weights='imagenet',
        input_tensor=None,
        input_shape=(150,150,3), 
        pooling=None,
        classes=1,
        classifier_activation="sigmoid",
    )
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    prediction = Dense(1, activation=tf.nn.sigmoid)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = get_model()
model.summary()


In [11]:
#datagen = ImageDataGenerator(rescale=1./255)
datagen = ImageDataGenerator()
# datagen.fit(x_train)
# type(x_train)
# datagen.fit(x_val)
# type(x_val)

In [ ]:
 for epoch in range(2):
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=16):
        #model.fit(x_batch, y_batch)
        learner = ktrain.get_learner(model, train_data=(x_batch, y_batch), val_data = (x_val, y_val))
        learner.autofit(0.001, 5, monitor='val_accuracy', reduce_on_plateau=5, reduce_factor=10, checkpoint_folder='F:\EfficientNetB0')

In [ ]:
#learner = ktrain.get_learner(model, train_data=datagen.flow(x_train, y_train), val_data = (x_val, y_val))

In [ ]:
# rebuild the model to train from scratch 
#learner.set_model(get_model()) 
    
# training using autofit
#learner.autofit(0.001, 200, monitor='val_accuracy', reduce_on_plateau=5, reduce_factor=10, checkpoint_folder='F:\EfficientNetB0')
#learner.autofit(0.005, 2) 